In [33]:
import os
import sys
while not os.getcwd().endswith('ml'):
    os.chdir('..')
sys.path.insert(0, os.getcwd())

In [34]:
import math
import copy
import cv2
import numpy as np
import random
from shutil import copyfile
from pathlib import Path
from matplotlib import pyplot as plt
from kaggle_problems.rosneft_proppant import helpers
from kaggle_problems.rosneft_proppant.helpers import *
helpers.circleContour = GrayCircleContour()

In [36]:
# def almost_equal(a, b, e):
#     return abs(a - b) < e

# def get_circle_diff(contour):
#     s = cv2.contourArea(contour) # pi * R ^ 2
#     p = cv2.arcLength(contour, True) # 2 * pi * R
    
#     s /= math.pi
#     p /= math.pi * 2
#     if (s + p * p < 1e-5):
#         return 1. # MAX
    
#     return abs(s - p * p) / (s + p * p)

# def is_circle(contour):
#     s = cv2.contourArea(contour)
#     if (s > 20):
#         return get_circle_diff(contour) < 0.1
#     else:
#         return False


# def get_almost_convex(contour):
#     convex = cv2.convexHull(contour)
#     origin_area = cv2.contourArea(contour)
#     conveex_area = cv2.contourArea(convex)
#     return origin_area / max(1, conveex_area)

# def is_almost_convex(contour):
#     return get_almost_convex(contour) >= 0.9
    

In [37]:
# def get_contour_color(img, contour):
#     if (len(contour) == 0):
#         return None
#     return img[contour[0][0][1], contour[0][0][0]]

# def get_contour_center(contour):
#     if (len(contour) == 0):
#         return None
#     x, y = 0, 0
#     for sell in contour:
#         x += sell[0][1]
#         y += sell[0][0]
#     return (y / len(contour), x / len(contour))


In [42]:
def process_bw_img(original_img, img_number):
    result_circles = []
    debug_dir = "{}/{}".format(DEBUG_IMG_DIR, img_number)
    os.mkdir(debug_dir) # create debug dir
    
    img_with_circle = copy.deepcopy(original_img)
    img = copy.deepcopy(original_img)

    gaussian_3 = cv2.GaussianBlur(img, (0, 0), 2.0)
    img = cv2.addWeighted(img, 1.5, gaussian_3, -0.5, 0, img)
    cv2.imwrite("{}/img.jpg".format(debug_dir), img)

    gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)

    th, threshed = cv2.threshold(gray, 50, 255, cv2.THRESH_BINARY_INV | cv2.THRESH_OTSU)

    it = 0
    for r in range(15, 2, -1):
        print('-' * 30)
        # Выделение кругов
        circles = cv2.HoughCircles(threshed,cv2.HOUGH_GRADIENT,0.5,minDist=r,
                                param1=200,param2=1,minRadius=r,maxRadius=r)
        circles = circles[0]
        print("Circle count: {}. r: {}".format(len(circles), r))
        filtered_circle = [(int(circle[0]), int(circle[1]), int(circle[2])) for circle in circles]

        filtered_circle = [circle for circle in filtered_circle \
        if is_convex_in_circle(threshed, circle[0], circle[1], circle[2])]

        print("After first filter: {}. r: {}".format(len(filtered_circle), r))

        filtered_circle = [circle for circle in filtered_circle \
            if is_border_white_circle(threshed, circle[0], circle[1], circle[2])]

        print("After second filter: {}. r: {}".format(len(filtered_circle), r))

        for circle in filtered_circle:
            cv2.circle(threshed,(circle[0],circle[1]),circle[2], 0, -1) # fill with 0 because of THRESH_BINARY_INV
            cv2.circle(original_img,(circle[0],circle[1]),circle[2],(255, 255, 255),-1)
            cv2.circle(img_with_circle,(circle[0],circle[1]),circle[2], get_random_color(),1)

        cv2.imwrite("{}/original_img_{}.jpg".format(debug_dir, it), original_img)
        cv2.imwrite("{}/img_with_circle_{}.jpg".format(debug_dir, it), img_with_circle)
        cv2.imwrite("{}/threshed_{}.jpg".format(debug_dir, it), threshed)
        it += 1
        result_circles.extend(filtered_circle)
        
    return result_circles

In [43]:
img_name = "6"
original_img = cv2.imread("{}/tmp/{}".format(DATA_DIR, img_name + ".png")) 

process_bw_img(original_img, img_name)


------------------------------
Circle count: 3081. r: 15
After first filter: 0. r: 15
After second filter: 0. r: 15
------------------------------
Circle count: 3314. r: 14
After first filter: 3. r: 14
After second filter: 3. r: 14
------------------------------
Circle count: 4137. r: 13
After first filter: 6. r: 13
After second filter: 6. r: 13
------------------------------
Circle count: 4576. r: 12
After first filter: 85. r: 12
After second filter: 85. r: 12
------------------------------
Circle count: 4355. r: 11
After first filter: 160. r: 11
After second filter: 160. r: 11
------------------------------
Circle count: 5896. r: 10
After first filter: 758. r: 10
After second filter: 758. r: 10
------------------------------
Circle count: 7696. r: 9
After first filter: 475. r: 9
After second filter: 475. r: 9
------------------------------
Circle count: 8654. r: 8
After first filter: 200. r: 8
After second filter: 199. r: 8
------------------------------
Circle count: 9493. r: 7
Afte

[(630, 1323, 14),
 (952, 890, 14),
 (326, 1755, 14),
 (723, 1910, 13),
 (315, 1723, 13),
 (678, 1239, 14),
 (523, 404, 14),
 (691, 276, 13),
 (627, 1751, 14),
 (195, 507, 12),
 (306, 1347, 12),
 (209, 1273, 12),
 (605, 1798, 12),
 (887, 1177, 12),
 (234, 1195, 12),
 (472, 1475, 12),
 (372, 1712, 12),
 (716, 1323, 13),
 (769, 1802, 12),
 (225, 1389, 12),
 (365, 712, 12),
 (507, 323, 12),
 (162, 881, 12),
 (318, 565, 12),
 (439, 346, 12),
 (692, 2046, 12),
 (812, 1276, 12),
 (959, 1380, 12),
 (574, 490, 13),
 (1032, 1523, 13),
 (578, 1920, 13),
 (497, 1977, 12),
 (972, 1621, 12),
 (469, 1890, 12),
 (452, 1513, 13),
 (472, 403, 12),
 (499, 628, 12),
 (639, 781, 13),
 (693, 2020, 12),
 (868, 1474, 12),
 (905, 1632, 12),
 (925, 739, 12),
 (994, 1203, 12),
 (1009, 1554, 13),
 (415, 1436, 12),
 (462, 1583, 12),
 (997, 1305, 12),
 (573, 807, 13),
 (867, 1392, 13),
 (376, 1686, 12),
 (408, 574, 12),
 (476, 1689, 12),
 (582, 467, 12),
 (630, 653, 12),
 (723, 1726, 12),
 (828, 1876, 13),
 (526, 2

In [39]:
all_img = [img for img in os.listdir(ORIGINAL_IMG_DIR) if img.endswith('.jpg') ]

for img_name in all_img:
    img_number, _ = img_name.split('.')
    original_img = cv2.imread("{}/{}".format(ORIGINAL_IMG_DIR, img_name)) 
    Path("{}/processing/bw_images".format(DATA_DIR)).mkdir(exist_ok=True, parents=True)
    Path("{}/processing/color_images".format(DATA_DIR)).mkdir(exist_ok=True, parents=True)
    
    if is_grey_img(original_img):
        copyfile("{}/{}".format(ORIGINAL_IMG_DIR, img_name), "{}/processing/bw_images/{}".format(DATA_DIR, img_name))
    else:
        copyfile("{}/{}".format(ORIGINAL_IMG_DIR, img_name), "{}/processing/color_images/{}".format(DATA_DIR, img_name))


KeyboardInterrupt: 